# download data

In [1]:
! curl \
    --header 'Host: data.gov.ua' \
    --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:108.0) Gecko/20100101 Firefox/108.0' \
    --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' \
    --header 'Accept-Language: en-US,en;q=0.5' \
    --referer 'https://data.gov.ua/dataset/470196d3-4e7a-46b0-8c0c-883b74ac65f0' \
    --cookie '_ga=GA1.3.2116747368.1671616277; _gid=GA1.3.145106975.1673376918; _gat=1' \
    --header 'Upgrade-Insecure-Requests: 1' \
    --header 'Sec-Fetch-Dest: document' \
    --header 'Sec-Fetch-Mode: navigate' \
    --header 'Sec-Fetch-Site: same-origin' \
    --header 'Sec-Fetch-User: ?1' 'https://data.gov.ua/dataset/8851831d-b5ce-4ca8-8685-eafbc3f57eca/resource/6cfff17e-84ac-4141-b0fd-89abb68e9f31/download/mvswantedbezvesti_1.json' \
    --output 'data/mvswantedbezvesti_1.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.5M  100 18.5M    0     0  1639k      0  0:00:11  0:00:11 --:--:-- 1564k--:--  0:00:12 1569k


In [2]:
! tree --gitignore --dirsfirst -C

.
├── data
│   └── mvswantedbezvesti_1.json
├── venv
├── location_extraction.py
├── project.ipynb
└── requirements.txt

2 directories, 4 files


In [3]:
! cat requirements.txt

altair==4.2.0
pandas==1.5.2


# Pipeline

In [118]:
import datetime
import json

import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Get Data

### Load data

In [5]:
with open("data/mvswantedbezvesti_1.json") as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data)
print(df.shape)
df.head()

(26050, 20)


,ID,OVD,CATEGORY,FIRST_NAME_U,LAST_NAME_U,MIDDLE_NAME_U,FIRST_NAME_R,LAST_NAME_R,MIDDLE_NAME_R,FIRST_NAME_E,LAST_NAME_E,MIDDLE_NAME_E,BIRTH_DATE,SEX,LOST_DATE,LOST_PLACE,ARTICLE_CRIM,RESTRAINT,CONTACT,PHOTOID
0,12280824,ЧЕЧЕЛІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО В...,БЕЗВІСНО ЗНИКЛИЙ,НАТАЛІЯ,БОЙКО,СЕРГІЇВНА,НАТАЛЬЯ,БОЙКО,СЕРГЕЕВНА,NATALIYA,BOIKO,SERHIIVNA,1984-10-22T00:00:00,ЖІНОЧА,2000-11-20T00:00:00,"ДНЕПРОПЕТРОВСКАЯ, КРАСНОГВАРДЕЙСКИЙ, ДНЕПРОПЕТ...",СТ.115 Ч.1,",","056-7708234,056-7700634",00405597
1,12280698,П'ЯТИХАТСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ЖОВТОВОДСЬКОГО...,БЕЗВІСНО ЗНИКЛИЙ,ВІКТОР,ГЛАГОВСЬКИЙ,ТАДЄЄВІЧ,ВИКТОР,ГЛАГОВСКИЙ,ТАДЕЕВИЧ,VIKTOR,HLAHOVSKYI,TADIEIEVICH,1974-08-21T00:00:00,ЧОЛОВІЧА,2001-01-28T00:00:00,"ДНЕПРОПЕТРОВСКАЯ, ПЯТИХАТСКИЙ, МИРОЛЮБОВКА",СТ.115 Ч.1,",","05651-32202,056-7700634",00405641
2,12280443,ПІВДЕННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ КАМ`ЯНСЬКОГО ВІДДІ...,БЕЗВІСНО ЗНИКЛИЙ,ОЛЕНА,ХВАЛОВА,ОЛЕКСАНДРІВНА,ЕЛЕНА,ХВАЛОВА,АЛЕКСАНДРОВНА,OLENA,KHVALOVA,OLEKSANDRIVNA,1983-07-09T00:00:00,ЖІНОЧА,1998-03-12T00:00:00,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОДЗЕРЖИНСК, ДНЕПРОДЗЕРЖ...",СТ.115 Ч.1,",","05692-72484,056-7700634",00405661
3,12281408,ДНІПРОВСЬКЕ РАЙОННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВ...,БЕЗВІСНО ЗНИКЛИЙ,ВОЛОДИМИР,ХАНДРИГА,ПЕТРОВИЧ,ВЛАДИМИР,ХАНДРИГА,ПЕТРОВИЧ,VOLODYMYR,KHANDRYHA,PETROVYCH,1962-10-13T00:00:00,ЧОЛОВІЧА,1992-06-03T00:00:00,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОПЕТРОВСК, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,",","056-7537704,056-7700634",00405667
4,12280819,ДНІПРОВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП В ДНІПРОПЕТРО...,БЕЗВІСНО ЗНИКЛИЙ,ІГОР,ДОНЕЦЬ,МИХАЙЛОВИЧ,ИГОРЬ,ДОНЕЦ,МИХАЙЛОВИЧ,IHOR,DONETS,MYKHAILOVYCH,1974-06-21T00:00:00,ЧОЛОВІЧА,1998-10-19T00:00:00,"ДНЕПРОПЕТРОВСКАЯ, КИРОВСКИЙ, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,",","0562-7781297,056-7700634",00405673


### Investigate data

In [6]:
df.RESTRAINT.unique()

array([',', 'НЕ ЗАСТОСОВУВАВСЯ,', 'НЕ ЗАСТОСОВУВАВСЯ, 05-NOV-05',
       ', 13-NOV-12', 'НЕ ЗАСТОСОВУВАВСЯ, 11-JUL-05',
       'НЕ ЗАСТОСОВУВАВСЯ, 19-DEC-05', 'НЕ ЗАСТОСОВУВАВСЯ, 27-JUN-06',
       'НЕ ЗАСТОСОВУВАВСЯ, 14-JUL-06', 'НЕ ЗАСТОСОВУВАВСЯ, 01-AUG-07',
       'НЕ ЗАСТОСОВУВАВСЯ, 10-NOV-07', 'НЕ ЗАСТОСОВУВАВСЯ, 21-JUL-05',
       'НЕ ЗАСТОСОВУВАВСЯ, 26-FEB-00', 'НЕ ЗАСТОСОВУВАВСЯ, 27-DEC-07',
       'НЕ ЗАСТОСОВУВАВСЯ, 05-SEP-07', 'НЕ ЗАСТОСОВУВАВСЯ, 01-MAR-06',
       'НЕ ЗАСТОСОВУВАВСЯ, 03-SEP-07', 'НЕ ЗАСТОСОВУВАВСЯ, 16-AUG-07',
       'НЕ ЗАСТОСОВУВАВСЯ, 07-JUN-07', 'НЕ ЗАСТОСОВУВАВСЯ, 27-NOV-04',
       'НЕ ЗАСТОСОВУВАВСЯ, 22-OCT-05', 'НЕ ЗАСТОСОВУВАВСЯ, 13-DEC-05',
       'НЕ ЗАСТОСОВУВАВСЯ, 29-MAY-07'], dtype=object)

In [7]:
df.CATEGORY.unique()

array(['БЕЗВІСНО ЗНИКЛИЙ',
       'БЕЗВІСНО ЗНИКЛИЙ, ЯКИЙ СТАВ ЖЕРТВОЮ КРИМІНАЛЬНОГО ПРАВОПОРУШЕННЯ'],
      dtype=object)

In [8]:
df.ARTICLE_CRIM.unique()

array(['СТ.115 Ч.1', 'СТ.146 Ч.1', 'СТ.110 Ч.3', 'СТ.110 Ч.2', '',
       'СТ.1-146 Ч.1', 'СТ.438 Ч.1', 'СТ.438 Ч.2', 'СТ.146 Ч.2',
       'СТ.115 Ч.2 П.1', 'СТ.110 Ч.1', 'СТ.120 Ч.1', 'СТ.115 Ч.2 П.2',
       'СТ.286 Ч.2', 'СТ.146 Ч.3', 'СТ.349 Ч.1',
       'СТ.1-146 Ч.1, СТ.438 Ч.1', 'СТ.2-110 Ч.3',
       'СТ.438 Ч.1, СТ.187 Ч.4', 'СТ.147 Ч.1', 'СТ.348 Ч.1',
       'СТ.438 Ч.1, СТ.146 Ч.2', 'СТ.438 Ч.2, СТ.110 Ч.3',
       'СТ.115 Ч.1, СТ.438 Ч.1', 'СТ.258 Ч.3', 'СТ.185 Ч.1', 'СТ.213 Ч.1',
       'СТ.115 Ч.1, СТ.146 Ч.1, СТ.1-146 Ч.1', 'СТ.332 Ч.1', 'СТ.119 Ч.1',
       'СТ.115 Ч.2 П.8', 'СТ.115 Ч.2 П.6', 'СТ.115 Ч.2 П.3', 'СТ.151 Ч.1',
       'СТ.1-114 Ч.1'], dtype=object)

### Clear

In [9]:
df.drop(columns=[
    'FIRST_NAME_U',
    'LAST_NAME_U',
    'MIDDLE_NAME_U',

    'FIRST_NAME_R',
    'LAST_NAME_R',
    'MIDDLE_NAME_R',

    'FIRST_NAME_E',
    'LAST_NAME_E',
    'MIDDLE_NAME_E'
], inplace=True)

df.drop(columns=["RESTRAINT", 'CATEGORY'], inplace=True)

In [10]:
len_before = len(df)
df = df[df.LOST_DATE.apply(lambda x: x < datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))]
df = df[df.BIRTH_DATE.apply(lambda x: x < datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))]
len_after = len(df)
print(f"Видалено {len_before - len_after} некоректних записи")
df.LOST_DATE = pd.to_datetime(df.LOST_DATE, format="%Y-%m-%dT%H:%M:%S", errors="coerce")
df.BIRTH_DATE = pd.to_datetime(df.BIRTH_DATE, format="%Y-%m-%dT%H:%M:%S", errors="coerce")


Видалено 24 некоректних записи


In [11]:
df["AGE"] = (df.LOST_DATE - df.BIRTH_DATE).apply(lambda x: x.days / 365)

In [12]:
df.head()

,ID,OVD,BIRTH_DATE,SEX,LOST_DATE,LOST_PLACE,ARTICLE_CRIM,CONTACT,PHOTOID,AGE
0,12280824,ЧЕЧЕЛІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО В...,1984-10-22,ЖІНОЧА,2000-11-20,"ДНЕПРОПЕТРОВСКАЯ, КРАСНОГВАРДЕЙСКИЙ, ДНЕПРОПЕТ...",СТ.115 Ч.1,"056-7708234,056-7700634",00405597,16.090411
1,12280698,П'ЯТИХАТСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ЖОВТОВОДСЬКОГО...,1974-08-21,ЧОЛОВІЧА,2001-01-28,"ДНЕПРОПЕТРОВСКАЯ, ПЯТИХАТСКИЙ, МИРОЛЮБОВКА",СТ.115 Ч.1,"05651-32202,056-7700634",00405641,26.457534
2,12280443,ПІВДЕННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ КАМ`ЯНСЬКОГО ВІДДІ...,1983-07-09,ЖІНОЧА,1998-03-12,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОДЗЕРЖИНСК, ДНЕПРОДЗЕРЖ...",СТ.115 Ч.1,"05692-72484,056-7700634",00405661,14.684932
3,12281408,ДНІПРОВСЬКЕ РАЙОННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВ...,1962-10-13,ЧОЛОВІЧА,1992-06-03,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОПЕТРОВСК, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,"056-7537704,056-7700634",00405667,29.660274
4,12280819,ДНІПРОВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП В ДНІПРОПЕТРО...,1974-06-21,ЧОЛОВІЧА,1998-10-19,"ДНЕПРОПЕТРОВСКАЯ, КИРОВСКИЙ, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,"0562-7781297,056-7700634",00405673,24.345205


### Regex creation for ODV

In [71]:
import re

"""
country -> region             -> city  -> area              -> sub_area
Україна -> львівська область  -> Львів -> франкіський район
"""
new_regex0 = "(?P<city>.*) (?:УПРАВЛІННЯ|ВІДДІЛЕННЯ|ВІДДІЛ)(?: ПОЛІЦІЇ)? {1,2}ГУ {0,1}НП (?:В|У) (?P<region>.*) ОБЛ"
new_regex1 = r"(?P<area1>.*) ?(?:УПРАВЛІННЯ|ВІДДІЛЕННЯ|ВІДДІЛ)(?: ПОЛІЦІЇ)? ?(?P<number>№\d)?(?: (?P<city1>.*) ВІДДІЛУ?)?(?: ПОЛІЦІЇ)? ГУ {0,1}НП (?:В|У|) ?(?P<region1>.*) (?:ОБЛ\.|ОБЛАСТІ)"
"""Examples for regex1: # Please use regex101.com to validate
1)
ЧЕЧЕЛІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО ВІДДІЛУ ГУНП В ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
П'ЯТИХАТСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ЖОВТОВОДСЬКОГО ВІДДІЛУ ГУНП в ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
2)
АВДІЇВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ПОКРОВСЬКОГО ВІДДІЛУ ПОЛІЦІЇ ГУНП В ДОНЕЦЬКІЙ ОБЛ.
ЛЕБЕДИНСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ СУМСЬКОГО РАЙОННОГО ВІДДІЛУ ПОЛІЦІЇ ГУНП В СУМСЬКІЙ ОБЛАСТІ
КІРОВОГРАДСЬКЕ РАЙОННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ КРОПИВНИЦЬКОГО ВІДДІЛУ ПОЛІЦІЇ ГУНП В КІРОВОГРАДСЬКІЙ ОБЛАСТІ
3)
ДНІПРОВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП В ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
4)
ЧЕЧЕЛІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО ВІДДІЛУ ГУНП В ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
П'ЯТИХАТСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ЖОВТОВОДСЬКОГО ВІДДІЛУ ГУНП в ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
ПІВДЕННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ КАМ`ЯНСЬКОГО ВІДДІЛУ ГУНП В ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
ДНІПРОВСЬКЕ РАЙОННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО ВІДДІЛУ ГУНП В ДНІПРОПЕТРОВСЬКІЙ ОБЛ.
5)
ВІДДІЛЕННЯ ПОЛІЦІЇ №2 ПОЛТАВСЬКОГО ВІДДІЛУ ГУНП В ПОЛТАВСЬКІЙ ОБЛАСТІ
6)
УСАТІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ  БІЛЯЇВСЬКОГО ВІДДІЛУ ГУНП ОДЕСЬКІЙ ОБЛАСТІ
7)
КОРАБЕЛЬНЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ХЕРСОНСЬКОГО ВІДДІЛУ ГУ НП В ХЕРСОНСЬКІЙ ОБЛАСТІ
"""
new_regex2 = r"(?P<area2>.*) (?:УПРАВЛІННЯ|ВІДДІЛЕННЯ|ВІДДІЛ) ПОЛІЦІЇ ГУНП В М. (?P<city2>.*)"
"""
ДЕСНЯНСЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ

ДНІПРОВСЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ
ДЕСНЯНСЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ
СОЛОМ’ЯНСЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ
ШЕВЧЕНКІВСЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ
ДАРНИЦЬКЕ УПРАВЛІННЯ ПОЛІЦІЇ ГУНП В М. КИЄВІ
"""
new_regex3 = r"(?:(?P<sub_area3>.*) ВІДДІЛЕННЯ ПОЛІЦІЇ )?(?P<area_name3>.*) ВІДДІЛУ? ПОЛІЦІЇ В МІСТІ (?P<city3>.*) ГУ {0,1}НП (?:В|У) (?P<region3>.*) ОБЛАСТІ"
"""
СУВОРОВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ В МІСТІ ОДЕСІ ГУНП В ОДЕСЬКІЙ ОБЛАСТІ
ХАДЖИБЕЇВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ СУВОРОВСЬКОГО ВІДДІЛУ ПОЛІЦІЇ В МІСТІ ОДЕСІ ГУНП В ОДЕСЬКІЙ ОБЛАСТІ
"""

new_regex4 = r"(?:(?P<area4>.*) РВ )?(?:(?P<city4>.*) (?:МВ|МУ) )?ГУМВСУ (?:В|У)? ?(?P<region4>.*) ОБЛ\."
"""
СЛОВ’ЯНОСЕРБСЬКИЙ РВ ГУМВСУ У ЛУГАНСЬКІЙ ОБЛ.
КІРОВСЬКИЙ МВ ГУМВСУ У ЛУГАНСЬКІЙ ОБЛ.
ЖОВТНЕВИЙ РВ ЛУГАНСЬКОГО МУ ГУМВСУ У ЛУГАНСЬКІЙ ОБЛ.

КУЙБИШЕВСЬКИЙ РВ ДОНЕЦЬКОГО МУ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
БУДЬОНІВСЬКИЙ РВ ДОНЕЦЬКОГО МУ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
ПРОЛЕТАРСЬКИЙ РВ ДОНЕЦЬКОГО МУ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
СОВЄТСЬКИЙ РВ МАКІЇВСЬКОГО МУ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
КІРОВСЬКИЙ РВ МАКІЇВСЬКОГО МУ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.

ЧЕРВОНОГВАРДІЙСЬКИЙ РВ МАКІЇВСЬКОГО МУ ГУМВСУ ДОНЕЦЬКОЇ ОБЛ.
ЦЕНТРАЛЬНО-МІСЬКИЙ РВ МАКІЇВСЬКОГО МУ ГУМВСУ ДОНЕЦЬКОЇ ОБЛ.
"""
# TODO: add rule if city4 is None: city4 = sub_area4
"""
for next examples:
СТАРОБЕШІВСЬКИЙ РВ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
АМВРОСІЇВСЬКИЙ РВ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.

ПЕРВОМАЙСЬКИЙ МВ ГУМВСУ У ЛУГАНСЬКІЙ ОБЛ.
КІРОВСЬКИЙ МВ ГУМВСУ В ДОНЕЦЬКІЙ ОБЛ.
"""

new_regex5 = r"(?P<city5>.*) (МВ|РВ|ПВМ|МРВ) ГУ {0,1}МВ {0,1}С УКРАЇНИ( В)? АР КР?ИМ"
"""
ЯЛТИНСЬКЕ МУ СІМФЕРОПОЛЬСЬКОГО МВ ГУМВС УКРАЇНИ В АР КИМ # тому в крим стоїть ?
"""

new_regex6 = r"ГОЛОВНЕ УПРАВЛІННЯ НАЦІОНАЛЬНОЇ ПОЛІЦІЇ (В|У) (?P<city6>.*) ОБЛАСТІ"
new_regex7 = r"ГОЛОВНЕ УПРАВЛІННЯ НАЦІОНАЛЬНОЇ ПОЛІЦІЇ В( АВТОНОМНІЙ РЕСПУБЛІЦІ КРИМ ТА)? М. (?P<city7>.*)"
"""
ГОЛОВНЕ УПРАВЛІННЯ НАЦІОНАЛЬНОЇ ПОЛІЦІЇ В М. КИЄВІ
"""

new_regex8 = r"(?P<area8>.*) РВ (?P<city8>.*) МУ"
new_regex9 = r"(?:.*) В М. (?P<city9>КИЄВІ)"
"""
    "ВІДДІЛ ПОЛІЦІЇ В РІЧКОВОМУ ПОРТУ КИЇВ ГУНП В М. КИЄВІ")
    "УПРАВЛІННЯ ПОЛІЦІЇ В МЕТРОПОЛІТЕНІ ГУНП В М. КИЄВІ")
    "ВІДДІЛ ПОЛІЦІЇ НА СТАНЦІЯХ ЗАЛІЗНИЧНОГО ТРАНСПОРТУ ГУНП В М. КИЄВІ")
"""
new_regex10 = \
    r"(:?" + r")|(:?".join(
        [
            r"(?P<city10_1>ЖДАНОВСЬКИЙ) МВВС (?P<region10_1>ДОНЕЦЬКОЇ) ОБЛ.",
            r"\(ВИКЛЮЧЕНО\) ЗЕЛЕНОДОЛЬСЬКЕ ВП (?P<city10_2>НІКОПОЛЬСЬКОГО) ВІДДІЛУ ГУНП В (?P<region10_2>ДНІПРОПЕТРОВСЬКІЙ) ОБЛ.",
            r"ВІДДІЛ ПОЛІЦІЇ ЗОНИ (?P<city10_3>ЧОРНОБИЛЬСЬКОЇ) АЕС ГУНП В (?P<region10_3>КИЇВСЬКІЙ) ОБЛАСТІ",
            r"(?P<city10_4>КЕРЧЕНСЬКЕ) ГВ АР Крим",
            r"ОВСТ ЦЕНТРАЛЬНОГО АЕРОПОРТУ М. (?P<city10_5>СІМФЕРОПОЛЬ) АР КРИМ",
            r"УКР УМВС В (?P<city10_6>ІВАНО-ФРАНКІВСЬКОЇ) ОБЛ.",
            r"ЦЕНТРАЛЬНО-МІСЬКИЙ РВ (?P<city10_7>ГОРЛІВСЬКОГО) МУ ГУМВСУ ДОНЕЦЬКОЇ ОБЛ",
        ]) + r")"

In [14]:
# Перевірка одного regex
# regex = new_regex8
# tmp_match = df[df.OVD.apply(lambda s: bool(re.match(regex, s, flags=re.IGNORECASE)))]
# tmp_match.OVD.unique()

### Regex test

In [77]:
l_regexes = [
    new_regex1,
    new_regex2,
    new_regex3,
    new_regex4,
    new_regex5,
    new_regex6,
    new_regex7,
    new_regex8,
    new_regex9,
    new_regex10,
]
regex = r"(:?" + \
        r")|(:?".join(
            l_regexes
        ) + r")"

not_match = df[df.OVD.apply(lambda s: not bool(re.match(regex, s, flags=re.IGNORECASE)))]
match = df[df.OVD.apply(lambda s: bool(re.match(regex, s, flags=re.IGNORECASE)))]

assert not len(not_match), f"Деякі рядки не заматчились під regex {len(not_match)=} {len(match)=}"
if len(not_match):
    print(not_match.OVD.unique())

### Розпаршування результатів

In [114]:
info = df.OVD.apply(lambda s: re.match(regex, s, flags=re.IGNORECASE))
data = []
with open('data.csv', 'w') as f, open("broken.csv", "w") as bf:
    f.write(f"index,area,city,region\n")
    bf.write("|".join(df.columns) + "\n")
    for index, match in info.items():
        group = match.groupdict()

        area = [group[k] for k in group if k.startswith("area") and group[k] is not None]
        area = (area and area[0]) or None
        city = [group[k] for k in group if k.startswith("city") and group[k] is not None]
        city = (city and city[0]) or None
        if group["city4"] is None and group["area4"] is not None:
            # this works ok
            city = group["area4"]
        if group["city1"] is None and group["region1"] is not None:
            city = group["region1"]

        region = [group[k] for k in group if k.startswith("region") and group[k] is not None]
        region = (region and region[0]) or None
        if not city:
            bf.write("|".join(map(str, df.loc[index].tolist())) + "\n")
        f.write(f"{index},{area},{city},{region}\n")

        data.append((region, city, area,))

df[["region", "city", "area"]] = data
df.head()

,ID,OVD,BIRTH_DATE,SEX,LOST_DATE,LOST_PLACE,ARTICLE_CRIM,CONTACT,PHOTOID,AGE,region,city,area
0,12280824,ЧЕЧЕЛІВСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВСЬКОГО В...,1984-10-22,ЖІНОЧА,2000-11-20,"ДНЕПРОПЕТРОВСКАЯ, КРАСНОГВАРДЕЙСКИЙ, ДНЕПРОПЕТ...",СТ.115 Ч.1,"056-7708234,056-7700634",00405597,16.090411,ДНІПРОПЕТРОВСЬКІЙ,ДНІПРОВСЬКОГО,ЧЕЧЕЛІВСЬКЕ
1,12280698,П'ЯТИХАТСЬКЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ЖОВТОВОДСЬКОГО...,1974-08-21,ЧОЛОВІЧА,2001-01-28,"ДНЕПРОПЕТРОВСКАЯ, ПЯТИХАТСКИЙ, МИРОЛЮБОВКА",СТ.115 Ч.1,"05651-32202,056-7700634",00405641,26.457534,ДНІПРОПЕТРОВСЬКІЙ,ЖОВТОВОДСЬКОГО,П'ЯТИХАТСЬКЕ
2,12280443,ПІВДЕННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ КАМ`ЯНСЬКОГО ВІДДІ...,1983-07-09,ЖІНОЧА,1998-03-12,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОДЗЕРЖИНСК, ДНЕПРОДЗЕРЖ...",СТ.115 Ч.1,"05692-72484,056-7700634",00405661,14.684932,ДНІПРОПЕТРОВСЬКІЙ,КАМ`ЯНСЬКОГО,ПІВДЕННЕ
3,12281408,ДНІПРОВСЬКЕ РАЙОННЕ ВІДДІЛЕННЯ ПОЛІЦІЇ ДНІПРОВ...,1962-10-13,ЧОЛОВІЧА,1992-06-03,"ДНЕПРОПЕТРОВСКАЯ, ДНЕПРОПЕТРОВСК, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,"056-7537704,056-7700634",00405667,29.660274,ДНІПРОПЕТРОВСЬКІЙ,ДНІПРОВСЬКОГО,ДНІПРОВСЬКЕ РАЙОННЕ
4,12280819,ДНІПРОВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП В ДНІПРОПЕТРО...,1974-06-21,ЧОЛОВІЧА,1998-10-19,"ДНЕПРОПЕТРОВСКАЯ, КИРОВСКИЙ, ДНЕПРОПЕТРОВСК",СТ.115 Ч.1,"0562-7781297,056-7700634",00405673,24.345205,ДНІПРОПЕТРОВСЬКІЙ,ДНІПРОПЕТРОВСЬКІЙ,ДНІПРОВСЬКИЙ


## Стартові питання до набору даних:
- В яких регіонах зафіксована найбільша кількість безвісно зниклих осіб?
- Як розподіляються безвісно зниклі за роком зникнення та регіоном?
- Як розподіляються безвісно зниклі за роком зникнення, регіоном та статтю?
- Як розподіляються безвісно зниклі за роком народження або віком на момент зникнення та статтю?
- Як розподіляються безвісно зниклі за роком або ж роком і місяцем зникнення та статтю?

In [140]:
import numpy as np

/tmp/ipykernel_9075/1541622306.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['AGE_CATEGORY'] = pd.cut(x=input_data.AGE, bins=bins, labels=labels, include_lowest=True)


24-50     16245
50-65      4284
14-24      2779
65-100     2401
0-14        316
Name: AGE_CATEGORY, dtype: int64

In [168]:
input_data = df[["AGE", "SEX"]]

bins = [0, 14, 24, 50, 65, np.ceil(df.AGE.max())]
labels = ['0-14', '14-24', '24-50', '50-65', f'65-{int(np.ceil(df.AGE.max()))}']
index = input_data.SEX == "ЖІНОЧА"
input_data.loc[index, 'AGE_CATEGORY'] = pd.cut(x=input_data[index].AGE, bins=bins, labels=labels, include_lowest=True)
input_data.loc[~index, 'AGE_CATEGORY'] = pd.cut(x=input_data[~index].AGE, bins=bins, labels=labels, include_lowest=True)
input_data = input_data.groupby(by=["SEX", 'AGE_CATEGORY'])["SEX", 'AGE_CATEGORY'].value_counts().reset_index() #.value_counts().reset_index()
input_data.rename(columns={0: "amount"}, inplace=True)

/tmp/ipykernel_9075/3572840981.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.loc[index, 'AGE_CATEGORY'] = pd.cut(x=input_data[index].AGE, bins=bins, labels=labels, include_lowest=True)
/tmp/ipykernel_9075/3572840981.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  input_data = input_data.groupby(by=["SEX", 'AGE_CATEGORY'])["SEX", 'AGE_CATEGORY'].value_counts().reset_index() #.value_counts().reset_index()


In [169]:
# Як розподіляються безвісно зниклі за роком народження або віком на момент зникнення та статтю?

alt.Chart(input_data).mark_bar().encode(
    x=alt.X("AGE_CATEGORY:O"),
    y=alt.Y("amount:Q"),
    column=alt.Column("SEX:N"),
    color=alt.Color("SEX:N"),
).properties(width=300, height=600)

/home/ahdrew/workspaces/study/visualization/final_project/venv/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)